# 处理得到各变量的history

In [2]:
from typing import Callable
import json
import gymnasium as gym
import numpy as np
import os
from tqdm import tqdm
# from stable_baselines3 import DDPG
# from stable_baselines3 import TD3
# from stable_baselines3 import PPO
# from stable_baselines3 import SAC
from sbx import DDPG, DQN, PPO, SAC, TD3, TQC, CrossQ
from utils.env_util import make_vec_env
from stable_baselines3.common.vec_env import VecNormalize
from gymnasium.envs.registration import register
import pandas as pd
import matplotlib.pyplot as plt
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
# get current directory of the current .ipynb 
cdir = os.path.abspath('.') # 获取当前目录
pardir = os.path.abspath(os.path.join(os.getcwd(), "../")) # 获取上级目录
dbdir = os.path.abspath(os.path.join(os.getcwd(), "../..")) # 获取上上级目录(为了读取美股数据库数据)

# 将变量处理为终身路径
def var_path(vars, eval_episodes, env):
    max_length = max(len(path) for path in vars)
    var_path_array = np.full((eval_episodes, max_length), np.nan)
    for i, path in enumerate(vars):
        var_path_array[i, :len(path)] = path
    age_array = np.arange(env.get_attr('tb')[0], env.get_attr('tb')[0] + max_length)
    full_var_path = np.column_stack((age_array, var_path_array.T))
    var_path_df = pd.DataFrame(full_var_path,
                                    columns=['年龄'] + [f'个体{i+1}' for i in range(eval_episodes)])
    return var_path_df

# import torch 
# torch.autograd.set_detect_anomaly(True)
import warnings
warnings.filterwarnings('ignore')


import sys
cdir = os.path.abspath('.') # 获取当前目录
# print(sys.path)
# 状态空间：现金余额，个人养老金账户余额，永久收入冲击，年龄
register(
    id='cocco-rl-norm',
    entry_point='cocco_env_rl_norm:CoccoEnv',
)
register(
    id='cocco-dp',
    entry_point='cocco_env_dp:CoccoEnv',
)
log_dir = 'models//cocco_sac'

eval_dir = 'eval//'
# model_name = ['v13_pensionfund_penlim0_run7',
#               'v13_pensionfund_penlim1p2_run15',
#               'v13_pensionfund_nopenlim_run7'
#               ]
rl_model_name = [
            #   'cocco-rl_cocco_run225//best_model_67', # 无随机死亡最佳
            # 'cocco-rl_cocco_run231//best_model_67', # 有随机死亡最佳 
            # 'cocco-rl_cocco_run248//best_model_567'
            'cocco-rl-norm_cocco_run112//best_model',
            # 'cocco-rl-norm_cocco_run11//best_model'
              ]
# model_name = [d for d in os.listdir(log_dir) if os.path.isdir(os.path.join(log_dir, d))]
eval_episodes = 10000
eval_seed =  3687851522
# eval_seed = 1000
params = {'distf': 0.95}



def evaluate_model_dp(eval_episodes=10000, eval_seed=3687851522):

    current_path = os.path.dirname(os.path.abspath(__file__))
    df_A = pd.read_excel(os.path.join(current_path,'result_cocco_matlab','A.xlsx'),header=None)
    df_C = pd.read_excel(os.path.join(current_path,'result_cocco_matlab','C.xlsx'),header=None) 
    df_V = pd.read_excel(os.path.join(current_path,'result_cocco_matlab','V.xlsx'),header=None)  
    df_gcash = pd.read_excel(os.path.join(current_path,'result_cocco_matlab','gcash.xlsx'),header=None)
    # 转换为numpy数组
    A = df_A.to_numpy()
    C = df_C.to_numpy()
    V = df_V.to_numpy()
    gcash = df_gcash.to_numpy().squeeze()

    env = make_vec_env("cocco-dp",seed=1, n_envs=1,  env_kwargs={'params': None})

    action_set = []
    score_history = []
    age = []
    income = []
    consum = []
    tmp_income = []
    temp_consum = []
    
    reward_history = []

    # from tqdm import tqdm
    pbar = tqdm(total=eval_episodes, desc=f'dp模拟进度')    
    for episode in range(eval_episodes):
        pbar.update(1)
        env.seed(seed=eval_seed+episode)
        obs = env.reset()    
        done = False
        score = 0
        episode_actions = np.empty((0,2))
        temp_reward = []
        while not done:
            # 根据state[0]获取现金
            cash = np.squeeze(obs[0][0])        
            age_loc =  int(obs[0][1]) - env.get_attr('tb')[0] # 获取年龄

            # 根据现金和年龄插值从A中获取action
            action = np.array([np.interp(cash, gcash, C[:, age_loc]),np.interp(cash, gcash, A[:, age_loc])])
            if age_loc == np.shape(C)[1]-1: # 如果到达最后一期,则固定消费比例为1（全部消费干净）
                action[0] = cash
            else:
                action[0] = np.clip(action[0],0,cash)
            # action[0] = action[0]/cash # 转换为消费比例
            action[1] = np.clip(action[1],0,1)


            obs, reward, done, info = env.step(np.array([action]))
            score += reward * (params['distf'] ** len(episode_actions))
            action[0] = action[0]/cash # 转换为消费比例
            episode_actions = np.vstack((episode_actions, action))
            temp_consum.append(info[0]['raw_consumption'])
            temp_reward.append(reward[0])
            # if info[0]['status']=='working':
            tmp_income.append(info[0]['raw_income'])

        age.append(info[0]['state'][1])
        score_history.append(score[0])
        action_set.append(episode_actions)
        reward_history.append(temp_reward)
        # norm_age = info[0]['state'][4]
        # age.append(np.round((norm_age + 1) * (params['max_age'] - params['born_age']) / 2 + params['born_age'], 1) - 1)
        

        consum.append(temp_consum)
        income.append(tmp_income)


        tmp_income = []
        temp_consum = []


    life_path_detailed = pd.DataFrame({
        '死亡年龄': age,
        '工作期平均收入(万)': income,
        '终身效用': score_history,
    })
    # 保存评估文件
    # 从模型名中提取标识符（第二个_之后的字符串）
    model_identifier = 'dp'

    os.makedirs(eval_dir + model_identifier + '//', exist_ok=True)

    # =====保存消费路径=====
    consum_path_df = var_path(consum, eval_episodes, env)
    consum_path_df.to_excel(eval_dir+ model_identifier + "//" + model_identifier + "_consum_history.xlsx", index=False)
   # =====保存奖励历史=====
    reward_history_df = var_path(reward_history, eval_episodes, env)
    reward_history_df.to_excel(eval_dir+ model_identifier + "//" + model_identifier + "_reward_history.xlsx", index=False)
    # 保存收入历史
    income_history_df = var_path(income, eval_episodes, env)
    income_history_df.to_excel(eval_dir+ model_identifier + "//" + model_identifier + "_income_history.xlsx", index=False)
    # 保存风险资产比例决策历史
    risky_pct = []
    for episode in range(eval_episodes):
        risky_pct.append(action_set[episode][:,1])
    risky_pct_history_df = var_path(risky_pct, eval_episodes, env)
    risky_pct_history_df.to_excel(eval_dir+ model_identifier + "//" + model_identifier + "_risky_pct_history.xlsx", index=False)

        # 保存终身效用
    utility_path = eval_dir + model_identifier + "//" + model_identifier + "_score_history.xlsx"
    utility_path_df = pd.DataFrame(score_history, columns=['终身实现效用'])
    utility_path_df.insert(0, 'agent', np.arange(1, eval_episodes+1)) # 增加第一列为agent
    utility_path_df.to_excel(eval_dir+ model_identifier + "//" + model_identifier + "_score_history.xlsx", index=False)

    life_summary = np.array([
        np.mean(age),
        np.mean(score_history),
        np.std(score_history),
    ])
    life_summary_df = pd.DataFrame([life_summary], 
                                columns=[ '死亡年龄', '终身效用','终身效用标准差'])
    print(model_identifier,': ',life_summary_df)


    

def evaluate_model_rl(idx, eval_episodes=1000, eval_seed=3687851522):
    # # with open(os.path.join(log_dir, idx) +"//params.json", 'r') as file:
    # #     params = json.load(file)
    # current_path = os.path.dirname(os.path.abspath(__file__))    
    # # env = make_vec_env("cocco-rl",seed=1, n_envs=16, env_kwargs={'params': None})
    # # env = VecNormalize.load(os.path.join(current_path, log_dir,idx) + "//vec_env.pkl", env)
    # model = SAC.load(os.path.join(current_path, log_dir, idx), env=None) # , print_system_info=True

    # env = make_vec_env("cocco-rl",seed=eval_seed, n_envs=1, env_kwargs={'params': None})
    # idx = idx.split('//')[0]
    # env = VecNormalize.load(os.path.join(current_path, log_dir, idx) + "//eval_env.pkl",env)
    # # env = make_vec_env("cocco-rl",seed=eval_seed, n_envs=1, env_kwargs={'params':None},\
    # #                         monitor_kwargs={'discount_factor': params['distf']})         
    # # env = VecNormalize(env, norm_obs=True, norm_reward=False)
    # # model = DDPG.load(os.path.join(log_dir, idx) + "//latest_model", env)

    current_path = os.path.dirname(os.path.abspath(__file__))     
   
    env = make_vec_env("cocco-rl-norm",seed=1, n_envs=1, env_kwargs={'params': None})
    env = VecNormalize.load(os.path.join(current_path, log_dir,idx.split('//')[0]) + "//best_model_vec_env.pkl", env)
    model = SAC.load(os.path.join(current_path, log_dir, idx), env=env)

    env = make_vec_env("cocco-rl-norm",seed=eval_seed, n_envs=1, env_kwargs={'params':None},\
                            monitor_kwargs={'discount_factor':params['distf']})         
    env = VecNormalize(env, norm_obs=False, norm_reward=False) # 评估不需要norm_obs


    idx = idx.split('//')[0]
    

    action_set = []
    score_history = []
    age = []
    income = []
    consum = []
    tmp_income = []
    temp_consum = []
    reward_history = []
    # from tqdm import tqdm
    pbar = tqdm(total=eval_episodes, desc=f'rl模拟进度')  
    for episode in range(eval_episodes):  
        pbar.update(1)
        env.seed(seed=eval_seed+episode)
        obs = env.reset()    
        done = False
        score = 0
        episode_actions = np.empty((0,2))
        temp_reward = []
        while not done:
            action, _states = model.predict(model.env.normalize_obs(obs), deterministic=True)
            # 如果为最后一期，则固定消费比例为1（全部消费干净）
            if len(episode_actions) == env.get_attr('tn')[0]-1:
                action[0] = 1
            # cash = env.unnormalize_obs(obs[0])[0] # 反归一化
            obs, reward, done, info = env.step(action)               
            # reward = -(((cash*action[0][0]))**(1-env.get_attr('gamma')[0]))*10 + 10       
            score += reward * (params['distf'] ** len(episode_actions))
            episode_actions = np.vstack((episode_actions, info[0]['real_actions']))
            temp_consum.append(info[0]['消费'])
            temp_reward.append(reward[0])
            tmp_income.append(info[0]['raw_income'])
        
        score_history.append(score)
        reward_history.append(temp_reward)
        action_set.append(episode_actions)
        # norm_age = info[0]['state'][4]
        # age.append(np.round((norm_age + 1) * (params['max_age'] - params['born_age']) / 2 + params['born_age'], 1) - 1)
        age.append(info[0]['state'][1])

        consum.append(temp_consum)
        income.append(tmp_income)


        tmp_income = []
        temp_consum = []


    life_path_detailed = pd.DataFrame({
        '死亡年龄': age,
        '工作期平均收入(万)': income,
        '终身效用': score_history,
    })
    # 保存评估文件
    # 从模型名中提取标识符（第二个_之后的字符串）
    model_identifier = 'rl_' +idx.split('_', 2)[2] 

    os.makedirs(eval_dir + model_identifier + '//', exist_ok=True)

# =====保存消费路径=====
    consum_path_df = var_path(consum, eval_episodes, env)
    consum_path_df.to_excel(eval_dir+ model_identifier + "//" + model_identifier + "_consum_history.xlsx", index=False)
    # 计算各个分位数
    # consum_path_df = consum_path_df.iloc[:,]
    # consum_stats = pd.DataFrame({
    #     '均值': consum_path_df.mean(),
    #     '0.1%': consum_path_df.quantile(0.001),
    #     '1%': consum_path_df.quantile(0.01),
    #     '5%': consum_path_df.quantile(0.05),
    #     '10%': consum_path_df.quantile(0.1),
    #     '25%': consum_path_df.quantile(0.25),
    #     '50%': consum_path_df.quantile(0.5),
    #     '75%': consum_path_df.quantile(0.75),
    #     '90%': consum_path_df.quantile(0.9),
    #     '95%': consum_path_df.quantile(0.95),
    #     '99%': consum_path_df.quantile(0.99),
    #     '99.9%': consum_path_df.quantile(0.999)
    # })
    # # 保存统计结果
    # print(consum_stats)
    
   # =====保存奖励历史=====
    reward_history_df = var_path(reward_history, eval_episodes, env)
    reward_history_df.to_excel(eval_dir+ model_identifier + "//" + model_identifier + "_reward_history.xlsx", index=False)
    # 保存收入历史
    income_history_df = var_path(income, eval_episodes, env)
    income_history_df.to_excel(eval_dir+ model_identifier + "//" + model_identifier + "_income_history.xlsx", index=False)
    # 保存风险资产比例决策历史
    risky_pct = []
    for episode in range(eval_episodes):
        risky_pct.append(action_set[episode][:,1])
    risky_pct_history_df = var_path(risky_pct, eval_episodes, env)
    risky_pct_history_df.to_excel(eval_dir+ model_identifier + "//" + model_identifier + "_risky_pct_history.xlsx", index=False)
     # 保存终身效用
    utility_path = eval_dir + model_identifier + "//" + model_identifier + "_score_history.xlsx"
    utility_path_df = pd.DataFrame(score_history, columns=['终身实现效用'])
    utility_path_df.insert(0, 'agent', np.arange(1, eval_episodes+1)) # 增加第一列为agent
    utility_path_df.to_excel(eval_dir+ model_identifier + "//" + model_identifier + "_score_history.xlsx", index=False)
    

    # # =====保存生命总结=====
    life_summary = np.array([
        np.mean(age),
        np.mean(score_history),
        np.std(score_history),
    ])
    

    life_summary_df = pd.DataFrame([life_summary], 
                                columns=[ '死亡年龄', '终身效用','终身效用标准差'])
    print(idx,': ',life_summary_df)
 

if __name__ == '__main__':
 
    # # 循环执行evaluate_model
    evaluate_model_dp(eval_episodes=eval_episodes, eval_seed=eval_seed)
    for model_name in rl_model_name:
        evaluate_model_rl(model_name, eval_episodes=eval_episodes, eval_seed=eval_seed)
    # evaluate_model_rl(rl_model_name[0], eval_episodes=eval_episodes, eval_seed=eval_seed)


NameError: name '__file__' is not defined

# RL优于DP样本的平均消费决策

In [ ]:
from typing import Callable
import json
import gymnasium as gym
import numpy as np
import os
# from stable_baselines3 import DDPG
# from stable_baselines3 import TD3
# from stable_baselines3 import PPO
# from stable_baselines3 import SAC
from sbx import DDPG, DQN, PPO, SAC, TD3, TQC, CrossQ
from utils.env_util import make_vec_env
from stable_baselines3.common.vec_env import VecNormalize
from gymnasium.envs.registration import register
import pandas as pd
import matplotlib.pyplot as plt
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
# get current directory of the current .ipynb 
cdir = os.path.abspath('.') # 获取当前目录
pardir = os.path.abspath(os.path.join(os.getcwd(), "../")) # 获取上级目录
dbdir = os.path.abspath(os.path.join(os.getcwd(), "../..")) # 获取上上级目录(为了读取美股数据库数据)

# import torch 
# torch.autograd.set_detect_anomaly(True)
import warnings
warnings.filterwarnings('ignore')


import sys
cdir = os.path.abspath('.') # 获取当前目录
# print(sys.path)
# 状态空间：现金余额，个人养老金账户余额，永久收入冲击，年龄

log_dir = 'models//cocco_sac'

eval_dir = 'eval//'
rl_model_name = [
            #   'cocco-rl_cocco_run225//best_model_67', # 无随机死亡最佳
            # 'cocco-rl_cocco_run231//best_model_67', # 有随机死亡最佳 
            # 'cocco-rl_cocco_run242//best_model_618'
            # 'cocco-rl-norm_cocco_run15//best_model',
            'cocco-rl-norm_cocco_run16//best_model'
              ]




if __name__ == '__main__':
        # 获取当前所在路径中"博士论文"所在目录
    current_path = os.getcwd()
    while os.path.basename(current_path) != "博士论文":
        current_path = os.path.dirname(current_path)
        if current_path == os.path.dirname(current_path):  # 已到达根目录仍未找到
            raise Exception("未找到'博士论文'目录")
    thesis_dir = current_path + "\\文字\\2-与传统方法比较\\fig\\"

    # =====读取dp和rl model的score_history======
    utility_path = {}
    model_identifier = 'rl_' + rl_model_name[0].split('_',2)[-1].split('//')[0]
    rl_path = eval_dir + model_identifier + "//" + model_identifier + "_score_history.xlsx"
    utility_path['RL'] = pd.read_excel(rl_path)
    model_identifier = 'dp'
    dp_path = eval_dir + model_identifier + "//" + model_identifier + "_score_history.xlsx"
    utility_path['DP'] = pd.read_excel(dp_path)

    # 找出RL中对应score比DP大的序号
    rl_better_idx = utility_path['RL']['终身实现效用'] > utility_path['DP']['终身实现效用']
    print(f"RL表现优于DP的样本数: {rl_better_idx.sum()}")
    print(f"RL表现优于DP的样本占比: {rl_better_idx.mean():.2%}")

    # =====读取dp和rl model的收入历史======
    income_path = {}
    model_identifier = 'rl_' + rl_model_name[0].split('_',2)[-1].split('//')[0]
    rl_path = eval_dir + model_identifier + "//" + model_identifier + "_income_history.xlsx"
    income_path['RL'] = pd.read_excel(rl_path)
    rl_rl_better = np.mean(income_path['RL'].iloc[:,1:].iloc[:,rl_better_idx.values])
    rl_dp_better = np.mean(income_path['RL'].iloc[:,1:].iloc[:,~rl_better_idx.values])


    print([f'RL更好的路径收入:{rl_rl_better}, DP更好的路径收入: {rl_dp_better}'])
    # =====读取dp和rl model的消费历史======
    # consum_path = {}
    # model_identifier = 'rl_' + rl_model_name[0].split('_',2)[-1].split('//')[0]
    # rl_path = eval_dir + model_identifier + "//" + model_identifier + "_consum_history.xlsx"
    # consum_path['RL'] = pd.read_excel(rl_path)
    # consum_path['RL'] = pd.concat([consum_path['RL'].iloc[:,0], consum_path['RL'].iloc[:,1:].iloc[:,rl_better_idx.values]], axis=1)
    # model_identifier = 'dp'
    # dp_path = eval_dir + model_identifier + "//" + model_identifier + "_consum_history.xlsx"
    # consum_path['DP'] = pd.read_excel(dp_path)
    # consum_path['DP'] = pd.concat([consum_path['DP'].iloc[:,0], consum_path['DP'].iloc[:,1:].iloc[:,rl_better_idx.values]], axis=1)

    # # 根据每个收入group画rl和dp的生涯消费决策与年龄的图
    # plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
    # plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号
    
    # age_list = consum_path['RL'].iloc[:,0]
    
    # # 计算每个年龄点上的平均消费
    # rl_mean = consum_path['RL'].iloc[:,1:].mean(axis=1,skipna=True)
    # dp_mean = consum_path['DP'].iloc[:,1:].mean(axis=1,skipna=True) 
    
    # plt.figure(figsize=(7,8))
    # plt.plot(age_list, rl_mean, label='强化学习')
    # plt.plot(age_list, dp_mean, label='动态规划')
    # plt.title('RL优于DP样本的平均消费决策')
    # plt.xlabel('年龄')
    # plt.ylabel('消费')
    # plt.legend()
    # plt.grid(True)
    # plt.tight_layout()
    
    # # 保存高清图到thesis_dir
    # plt.savefig(thesis_dir + '//better_group_consumption.png', dpi=500, bbox_inches='tight')
    # plt.show()






    


# 每期score分布情况：确定reward怎么设置

In [ ]:
from typing import Callable
import json
import gymnasium as gym
import numpy as np
import os
# from stable_baselines3 import DDPG
# from stable_baselines3 import TD3
# from stable_baselines3 import PPO
# from stable_baselines3 import SAC
from sbx import DDPG, DQN, PPO, SAC, TD3, TQC, CrossQ
from utils.env_util import make_vec_env
from stable_baselines3.common.vec_env import VecNormalize
from gymnasium.envs.registration import register
import pandas as pd
import matplotlib.pyplot as plt
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
# get current directory of the current .ipynb 
cdir = os.path.abspath('.') # 获取当前目录
pardir = os.path.abspath(os.path.join(os.getcwd(), "../")) # 获取上级目录
dbdir = os.path.abspath(os.path.join(os.getcwd(), "../..")) # 获取上上级目录(为了读取美股数据库数据)
plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签  
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号
# import torch 
# torch.autograd.set_detect_anomaly(True)
import warnings
warnings.filterwarnings('ignore')


import sys
cdir = os.path.abspath('.') # 获取当前目录
# print(sys.path)
# 状态空间：现金余额，个人养老金账户余额，永久收入冲击，年龄

log_dir = 'models//cocco_sac'

eval_dir = 'eval//'
rl_model_name = [
            #   'cocco-rl_cocco_run225//best_model_67', # 无随机死亡最佳
            # 'cocco-rl_cocco_run231//best_model_67', # 有随机死亡最佳 
            # 'cocco-rl_cocco_run242//best_model_618'
            'cocco-rl-norm_cocco_run11//best_model'
              ]




if __name__ == '__main__':
        # 获取当前所在路径中"博士论文"所在目录
    current_path = os.getcwd()
    while os.path.basename(current_path) != "博士论文":
        current_path = os.path.dirname(current_path)
        if current_path == os.path.dirname(current_path):  # 已到达根目录仍未找到
            raise Exception("未找到'博士论文'目录")
    thesis_dir = current_path + "\\文字\\2-与传统方法比较\\fig\\"

    # =====读取dp和rl model的score_history======
    gamma = 3.84
    utility_path = {}
    income_path = {}
    model_identifier = 'dp'
    dp_path = eval_dir + model_identifier + "//" + model_identifier + "_income_history.xlsx"
    income_path['DP'] = pd.read_excel(dp_path)

    dp_path = eval_dir + model_identifier + "//" + model_identifier + "_consum_history.xlsx"
    utility_path['DP'] = pd.read_excel(dp_path)
    # utility_path['DP'].iloc[:,1:] = utility_path['DP'].iloc[:,1:]**(1-3.84)/(1-3.84)
    

    # temp[temp<-5] = -5


    # 创建包含两个子图的图形
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 6))

    # 子图1：绘制收入直方图
    # 剔除nan值
    temp = income_path['DP'].iloc[:,1:].values
    temp = temp[~np.isnan(temp)]
    ax1.hist(temp, bins=200, weights=np.ones(len(temp)) / len(temp), alpha=0.7, color='r', label='概率分布')
    ax1.set_xlabel('收入值', fontsize=12)
    ax1.set_ylabel('出现比例', fontsize=12)
    ax1.set_title('动态规划模型收入的概率分布', fontsize=14)
    ax1.grid(True, alpha=0.3)
    ax1.legend(fontsize=10)

    
    # 子图2：绘制消费直方图  
    # 剔除nan值
    temp = utility_path['DP'].iloc[:,1:].values
    temp = temp[~np.isnan(temp)]
    ax2.hist(temp, bins=200, weights=np.ones(len(temp)) / len(temp), alpha=0.7, color='r', label='概率分布')
    ax2.set_xlabel('消费值', fontsize=12)
    ax2.set_ylabel('出现比例', fontsize=12)
    ax2.set_title('动态规划模型消费的概率分布', fontsize=14)
    ax2.grid(True, alpha=0.3)
    ax2.legend(fontsize=10)

    # 子图3：绘制效用直方图
    utility_path['DP'].iloc[:,1:] = np.log(utility_path['DP'].iloc[:,1:]) # 需要调试的效用函数
    temp = utility_path['DP'].iloc[:,1:].values
    temp = temp[~np.isnan(temp)]
    # temp = temp[temp>-3]
    # temp[temp<-2]=-2
    
    ax3.hist(temp, bins=200, weights=np.ones(len(temp)) / len(temp), alpha=0.7, color='b', label='概率分布')
    ax3.set_xlabel('效用值', fontsize=12)
    ax3.set_ylabel('出现比例', fontsize=12)
    ax3.set_title('动态规划模型效用的概率分布', fontsize=14)
    ax3.grid(True, alpha=0.3)
    ax3.legend(fontsize=10)

    plt.tight_layout()
    plt.show()


# 各收入段消费情况比较


In [ ]:
from typing import Callable
import json
import gymnasium as gym
import numpy as np
import os
# from stable_baselines3 import DDPG
# from stable_baselines3 import TD3
# from stable_baselines3 import PPO
# from stable_baselines3 import SAC
from sbx import DDPG, DQN, PPO, SAC, TD3, TQC, CrossQ
from utils.env_util import make_vec_env
from stable_baselines3.common.vec_env import VecNormalize
from gymnasium.envs.registration import register
import pandas as pd
import matplotlib.pyplot as plt
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
# get current directory of the current .ipynb 
cdir = os.path.abspath('.') # 获取当前目录
pardir = os.path.abspath(os.path.join(os.getcwd(), "../")) # 获取上级目录
dbdir = os.path.abspath(os.path.join(os.getcwd(), "../..")) # 获取上上级目录(为了读取美股数据库数据)

# import torch 
# torch.autograd.set_detect_anomaly(True)
import warnings
warnings.filterwarnings('ignore')


import sys
cdir = os.path.abspath('.') # 获取当前目录
# print(sys.path)
# 状态空间：现金余额，个人养老金账户余额，永久收入冲击，年龄
log_dir = 'models//cocco_sac'

eval_dir = 'eval//'
# model_name = ['v13_pensionfund_penlim0_run7',
#               'v13_pensionfund_penlim1p2_run15',
#               'v13_pensionfund_nopenlim_run7'
#               ]
rl_model_name = [
            #   'cocco-rl_cocco_run225//best_model_67', # 无随机死亡最佳
            #   'cocco-rl_cocco_run229//best_model_68',
            #   'cocco-rl_cocco_run230//best_model_63', # 有随机死亡最佳 
            # 'cocco-rl_cocco_run231//best_model_67', # 有随机死亡最佳 
            # 'cocco-rl_cocco_run233//best_model_141', # 有随机死亡最佳 
            # 'cocco-rl_cocco_run239//best_model_613',
            # 'cocco-rl_cocco_run240//best_model_418',
            # 'cocco-rl_cocco_run241//best_model_526',
            'cocco-rl-norm_cocco_run7//best_model'
            # 'cocco-rl_cocco_run234//best_model_131',
              ]
# model_name = [d for d in os.listdir(log_dir) if os.path.isdir(os.path.join(log_dir, d))]
eval_episodes = 1000
eval_seed =  3687851522
# eval_seed = 1000
params = {'distf': 0.95}



if __name__ == '__main__':

    # 获取当前所在路径中"博士论文"所在目录
    current_path = os.getcwd()
    while os.path.basename(current_path) != "博士论文":
        current_path = os.path.dirname(current_path)
        if current_path == os.path.dirname(current_path):  # 已到达根目录仍未找到
            raise Exception("未找到'博士论文'目录")
    thesis_dir = current_path + "\\文字\\2-与传统方法比较\\fig\\"
    

    # =====读取dp和rl model的score_history======
    consum_path = {}
    model_identifier = 'rl_' + rl_model_name[0].split('_',2)[-1].split('//')[0]
    rl_path = eval_dir + model_identifier + "//" + model_identifier + "_consum_history.xlsx"
    consum_path['RL'] = pd.read_excel(rl_path)
    model_identifier = 'dp'
    dp_path = eval_dir + model_identifier + "//" + model_identifier + "_consum_history.xlsx"
    consum_path['DP'] = pd.read_excel(dp_path)

    # =====读取dp model的income_history(与rl相同)======
    model_identifier = 'rl_' + rl_model_name[0].split('_',2)[-1].split('//')[0]
    rl_path = eval_dir + model_identifier + "//" + model_identifier + "_income_history.xlsx"
    income_path = pd.read_excel(rl_path)
    mean_income = income_path.iloc[:, 1:].mean(axis=0, skipna=True)
    # 获取收入分别处于0-10%、10-25%、25-50%、50-75%、75-90%，90-100%分位数的agent loc
    percentiles = [0, 10, 25, 50, 75, 90, 100]
    income_percentiles = np.percentile(mean_income, percentiles)
    income_groups = {}
    for i in range(len(percentiles)-1):
        if i == len(percentiles)-2:
            mask = mean_income >= income_percentiles[i]
        else:
            mask = (mean_income >= income_percentiles[i]) & (mean_income < income_percentiles[i+1])
        income_groups[f'{percentiles[i]}-{percentiles[i+1]}'] = np.where(mask)[0]
    # 根据以上loc,获取utility_path中对应loc的终身效用

    # 根据每个收入group画rl和dp的生涯消费决策与年龄的图,每个收入group一个子图
    # 添加画图的中文支持
    plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
    
    plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号
    idx = 0
    age_list = consum_path['RL'].iloc[:,0]
    plt.figure(figsize=(7,8))
    subplot_num = len(income_groups)
    row = (subplot_num + 1) // 2  # 向上取整
    col = 2
    for i, (group, loc) in enumerate(income_groups.items(), 1):
        plt.subplot(row, col, i)
        for model in ['RL', 'DP']:
            temp = consum_path[model].iloc[:,loc+1].mean(axis=1)
            plt.plot(age_list, temp, label=model)
        plt.title(f'收入分位: {group}%')
        plt.xlabel('年龄')
        plt.ylabel('消费')
        plt.legend()
    plt.tight_layout()
    # 保存高清图到thesis_dir
    plt.savefig(thesis_dir + '//income_group_consumption.png', dpi=500, bbox_inches='tight')
    
    plt.show()





# 各收入阶段agent已实现平均效用比较


In [8]:
from typing import Callable
import json
import gymnasium as gym
import numpy as np
import os
# from stable_baselines3 import DDPG
# from stable_baselines3 import TD3
# from stable_baselines3 import PPO
# from stable_baselines3 import SAC
from sbx import DDPG, DQN, PPO, SAC, TD3, TQC, CrossQ
from utils.env_util import make_vec_env
from stable_baselines3.common.vec_env import VecNormalize
from gymnasium.envs.registration import register
import pandas as pd
import matplotlib.pyplot as plt
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
# get current directory of the current .ipynb 
cdir = os.path.abspath('.') # 获取当前目录
pardir = os.path.abspath(os.path.join(os.getcwd(), "../")) # 获取上级目录
dbdir = os.path.abspath(os.path.join(os.getcwd(), "../..")) # 获取上上级目录(为了读取美股数据库数据)

# import torch 
# torch.autograd.set_detect_anomaly(True)
import warnings
warnings.filterwarnings('ignore')


import sys
cdir = os.path.abspath('.') # 获取当前目录
# print(sys.path)
# 状态空间：现金余额，个人养老金账户余额，永久收入冲击，年龄

log_dir = 'models//cocco_sac'

eval_dir = 'eval//'
rl_model_name = [
            #   'cocco-rl_cocco_run225//best_model_67', # 无随机死亡最佳
            # 'cocco-rl_cocco_run231//best_model_67', # 有随机死亡最佳 
            # 'cocco-rl_cocco_run242//best_model_618'
            'cocco-rl-norm_cocco_run21//best_model'
              ]




if __name__ == '__main__':


    # =====读取dp和rl model的score_history======
    utility_path = {}
    model_identifier = 'rl_' + rl_model_name[0].split('_',2)[-1].split('//')[0]
    rl_path = eval_dir + model_identifier + "//" + model_identifier + "_score_history.xlsx"
    utility_path['RL'] = pd.read_excel(rl_path)
    model_identifier = 'dp'
    dp_path = eval_dir + model_identifier + "//" + model_identifier + "_score_history.xlsx"
    utility_path['DP'] = pd.read_excel(dp_path)

    # =====读取dp model的income_history(与rl相同)======
    model_identifier = 'rl_' + rl_model_name[0].split('_',2)[-1].split('//')[0]
    rl_path = eval_dir + model_identifier + "//" + model_identifier + "_income_history.xlsx"
    income_path = pd.read_excel(rl_path)
    mean_income = income_path.iloc[:, 1:].mean(axis=0, skipna=True)
    # 获取收入分别处于0-10%、10-25%、25-50%、50-75%、75-90%，90-100%分位数的agent loc
    percentiles = [0, 10, 25, 50, 75, 90, 100]
    income_percentiles = np.percentile(mean_income, percentiles)
    income_groups = {}
    for i in range(len(percentiles)-1):
        if i == len(percentiles)-2:
            mask = mean_income >= income_percentiles[i]
        else:
            mask = (mean_income >= income_percentiles[i]) & (mean_income < income_percentiles[i+1])
        income_groups[f'{percentiles[i]}-{percentiles[i+1]}'] = np.where(mask)[0]
    # 根据以上loc,获取utility_path中对应loc的终身效用
    mean_utility = pd.DataFrame(columns=['模型', '收入分位(\%)', 'Mean','Std.','Median','Max','Min','Obs.'])
    idx = 0
    for model in ['RL', 'DP']:
        for group, loc in income_groups.items():
            temp = utility_path[model].iloc[loc, 1]
            income_mean = mean_income[loc].mean()  # 计算该收入分位的平均收入
            mean_utility.loc[idx,['模型', '收入分位(\%)',  'Mean','Std.','Median','Max','Min','Obs.']] = \
            [model, group,  temp.mean(), temp.std(), temp.median(), temp.max(), temp.min(), len(temp)]
            idx += 1
        temp = utility_path[model].iloc[:, 1]
        income_mean = mean_income.mean()  # 计算总体的平均收入
        mean_utility.loc[idx,['模型', '收入分位(\%)',  'Mean','Std.','Median','Max','Min','Obs.']] = \
            [model, 'all', temp.mean(), temp.std(), temp.median(), temp.max(), temp.min(), len(temp)]
        idx += 1


    # 输出为tex格式，保留小数点后两位
    f = mean_utility.to_latex(index=False, float_format='%.3f')
    # 将{lll...}替换为{>{\centering\arraybackslash}m{1cm}>{\centering\arraybackslash}m{3cm}*{列数-1}{>{\centering\arraybackslash}m{2cm}}},注意
    # 获取列数
    num_cols = len(mean_utility.columns)
    # 生成对应数量的列格式
    col_format = '{' + '>{\centering\\arraybackslash}m{2cm}' + '>{\centering\\arraybackslash}m{3cm}' + \
                 ''.join(['>{\centering\\arraybackslash}m{2cm}' for _ in range(num_cols-2)]) + '}'
    # 替换原有的列格式
    f = f.replace('{' + 'l'*num_cols + '}', col_format)
    print(f)
    


\begin{tabular}{>{\centering\arraybackslash}m{2cm}>{\centering\arraybackslash}m{3cm}>{\centering\arraybackslash}m{2cm}>{\centering\arraybackslash}m{2cm}>{\centering\arraybackslash}m{2cm}>{\centering\arraybackslash}m{2cm}>{\centering\arraybackslash}m{2cm}>{\centering\arraybackslash}m{2cm}}
\toprule
模型 & 收入分位(\%) & Mean & Std. & Median & Max & Min & Obs. \\
\midrule
RL & 0-10 & -18.993 & 6.161 & -17.568 & -9.208 & -45.462 & 1000 \\
RL & 10-25 & -8.932 & 2.321 & -8.530 & -4.170 & -26.019 & 1500 \\
RL & 25-50 & -4.822 & 1.451 & -4.595 & -0.339 & -18.917 & 2500 \\
RL & 50-75 & -2.790 & 0.952 & -2.625 & -0.147 & -8.838 & 2500 \\
RL & 75-90 & -1.900 & 0.745 & -1.742 & -0.613 & -6.747 & 1500 \\
RL & 90-100 & -1.263 & 0.561 & -1.151 & -0.248 & -4.952 & 1000 \\
RL & all & -5.553 & 5.579 & -3.596 & -0.147 & -45.462 & 10000 \\
DP & 0-10 & -21.541 & 8.818 & -19.447 & -4.907 & -64.550 & 1000 \\
DP & 10-25 & -8.972 & 2.758 & -8.525 & -3.140 & -28.887 & 1500 \\
DP & 25-50 & -4.686 & 1.515 & -4.411 & -

# 根据收入确定收入缩放比例（用于养老模型计算税收）

In [ ]:
from typing import Callable
import json
import gymnasium as gym
import numpy as np
import os
# from stable_baselines3 import DDPG
# from stable_baselines3 import TD3
# from stable_baselines3 import PPO
# from stable_baselines3 import SAC
from sbx import DDPG, DQN, PPO, SAC, TD3, TQC, CrossQ
from utils.env_util import make_vec_env
from stable_baselines3.common.vec_env import VecNormalize
from gymnasium.envs.registration import register
import pandas as pd
import matplotlib.pyplot as plt
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
# get current directory of the current .ipynb 
cdir = os.path.abspath('.') # 获取当前目录
pardir = os.path.abspath(os.path.join(os.getcwd(), "../")) # 获取上级目录
dbdir = os.path.abspath(os.path.join(os.getcwd(), "../..")) # 获取上上级目录(为了读取美股数据库数据)

# import torch 
# torch.autograd.set_detect_anomaly(True)
import warnings
warnings.filterwarnings('ignore')


import sys
cdir = os.path.abspath('.') # 获取当前目录
# print(sys.path)
# 状态空间：现金余额，个人养老金账户余额，永久收入冲击，年龄
log_dir = 'models//cocco_sac'

eval_dir = 'eval//'
# model_name = ['v13_pensionfund_penlim0_run7',
#               'v13_pensionfund_penlim1p2_run15',
#               'v13_pensionfund_nopenlim_run7'
#               ]
rl_model_name = [
            #   'cocco-rl_cocco_run225//best_model_67', # 无随机死亡最佳
            #   'cocco-rl_cocco_run229//best_model_68',
            #   'cocco-rl_cocco_run230//best_model_63', # 有随机死亡最佳 
            # 'cocco-rl_cocco_run231//best_model_67', # 有随机死亡最佳 
            # 'cocco-rl_cocco_run233//best_model_141', # 有随机死亡最佳 
            # 'cocco-rl_cocco_run239//best_model_613',
            # 'cocco-rl_cocco_run240//best_model_418',
            # 'cocco-rl_cocco_run241//best_model_526',
            'cocco-rl-norm_cocco_run7//best_model'
            # 'cocco-rl_cocco_run234//best_model_131',
              ]
# model_name = [d for d in os.listdir(log_dir) if os.path.isdir(os.path.join(log_dir, d))]
eval_episodes = 1000
eval_seed =  3687851522
# eval_seed = 1000
params = {'distf': 0.95}



if __name__ == '__main__':


    # =====读取dp和rl model的score_history======
    income_path = {}
    model_identifier = 'dp'
    dp_path = eval_dir + model_identifier + "//" + model_identifier + "_income_history.xlsx"
    income_path['DP'] = pd.read_excel(dp_path)

    # 计算20-60岁平均收入
    age_range = range(20, 61)  # 20-60岁
    dp_income = income_path['DP']
    
    # 筛选20-60岁数据
    dp_income_filtered = dp_income[dp_income['年龄'].isin(age_range)]
    
    # 计算平均收入
    temp = dp_income_filtered.iloc[:,1:].copy()
    scale = 6.87478
    scaled_temp = np.exp(np.log(temp)*scale)
    print(f"DP模型20-60岁收入缩放比例: {scale:.6f}")
    print(f"DP模型20-60岁平均收入: {np.mean(scaled_temp):.6f}万元")
    print(f"DP模型20-60岁最低收入: {np.min(scaled_temp):.6f}万元")
    print(f"DP模型20-60岁最高收入: {np.max(scaled_temp):.6f}万元")

    



